In [1]:
import inspect
import re
import random
import time
from typing import Any, Callable, Awaitable, List, TypeVar
from pathlib import Path
from dotenv import dotenv_values
import asyncio
import nest_asyncio
import pandas as pd
from pydantic import BaseModel, SecretStr, Field
from openai import OpenAI, AsyncOpenAI, RateLimitError
from openai.types.chat import ChatCompletion
import src
#from src.components.promptrunner import ResponseClient
import src.components.prompteval as pe
from src.prj_logger import ProjectLogger, get_logs
from src import utils
T = TypeVar("T")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Load config settings
DOT_ENV = dotenv_values("../.env")
openai_api_key = str(DOT_ENV['OPENAI_API_KEY'])
openai_api_key_secret = SecretStr(openai_api_key)
config = utils.load_config("../config.yaml")

# Create a unique run-id folder to store outputs
config["FILE_LOCATIONS"]["MAIN_DATA_FOLDER"] = "../src/data"
output_directory = utils.make_output_directory(config["FILE_LOCATIONS"])

# Instantiate logger
logger = ProjectLogger(
    name=src.BASE_LOGGERNAME,
    log_file=f"{output_directory}/{src.BASE_LOGGERNAME}.log"
)
logger.config()

In [3]:
async def async_retry_with_backoff(
    func: Callable[..., Awaitable[T]],
    *args,
    initial_delay=1,
    factor=2,
    jitter=True,
    max_retries=5,
    **kwargs
) -> T:
    delay = initial_delay
    for i in range(max_retries):
        try:
            return await func(*args, **kwargs)
        except Exception as e:  # OpenAI RateLimitError or generic AI API exception
            # You may want to customize this to check specific error types
            if i == max_retries - 1:
                raise
            sleep_time = delay * (1 + (random.random() if jitter else 0))
            print(f"Rate limited, retry attempt {i+1}/{max_retries}, sleeping {sleep_time:.1f}s...")
            await asyncio.sleep(sleep_time)
            delay *= factor
    raise Exception("Max retries exceeded.")

In [4]:
nest_asyncio.apply()
loop = asyncio.ProactorEventLoop()
asyncio.set_event_loop(loop)

In [5]:
class OpenAIRateLimiter:
    """Rate limiter for OpenAI API to stay under 500 requests per minute"""
    
    def __init__(self, max_requests_per_minute: int = 490):  # Buffer of 10 under the limit
        self.max_requests = max_requests_per_minute
        self.request_timestamps = []
        self.lock = asyncio.Lock()
    
    async def wait_if_needed(self):
        """Wait if we're approaching the rate limit"""
        async with self.lock:
            # Remove timestamps older than 1 minute
            current_time = time.time()
            self.request_timestamps = [ts for ts in self.request_timestamps 
                                      if current_time - ts < 60]
            
            # If we're at capacity, wait until we can make another request
            if len(self.request_timestamps) >= self.max_requests:
                oldest_timestamp = min(self.request_timestamps)
                wait_time = 60 - (current_time - oldest_timestamp) + 0.1  # Add a small buffer
                if wait_time > 0:
                    print(f"Rate limit approaching, waiting {wait_time:.2f}s...")
                    await asyncio.sleep(wait_time)
                    # After waiting, clean up timestamps again
                    current_time = time.time()
                    self.request_timestamps = [ts for ts in self.request_timestamps 
                                             if current_time - ts < 60]
            
            # Record this request
            self.request_timestamps.append(time.time())


async def async_retry_with_backoff(
    func: Callable[..., Awaitable[T]],
    *args: Any,
    initial_delay: float = 1,
    factor: float = 2,
    jitter: bool = True,
    max_retries: int = 5,
    **kwargs: Any
) -> T:
    """
    Async retry decorator with exponential backoff for OpenAI API calls
    """
    delay = initial_delay
    
    for i in range(max_retries):
        try:
            return await func(*args, **kwargs)
        except RateLimitError as e:
            if i == max_retries - 1:
                raise Exception(f"Max retries exceeded: {e}")
            
            sleep_time = delay * (1 + (random.random() if jitter else 0))
            print(f"Rate limited, retrying in {sleep_time:.1f}s... (Attempt {i+1}/{max_retries})")
            await asyncio.sleep(sleep_time)
            delay *= factor
    
    raise Exception("Max retries exceeded.")


class RateLimitOpenAIClient:
    """Wrapper for AsyncOpenAI client with rate limiting and retries"""
    
    def __init__(self, api_key: str = 'None', max_requests_per_minute: int = 490):
        self.client = AsyncOpenAI(api_key=api_key)
        self.rate_limiter = OpenAIRateLimiter(max_requests_per_minute)
    
    async def chat_completion(self, model: str, messages: list, **kwargs) -> ChatCompletion:
        """Make a chat completion request with rate limiting and retries"""
        await self.rate_limiter.wait_if_needed()
        
        return await async_retry_with_backoff(
            self.client.chat.completions.create,
            model=model,
            messages=messages,
            **kwargs
        )




In [30]:
# Evaluation functions

# Get all functions in the prompteval module
functions = inspect.getmembers(pe, inspect.isfunction)

exclude_funcs = [
    'eval_explicit_enumeration',
    'eval_follows_style_guide',
    'eval_has_correct_grammar',
    'eval_has_supporting_diagram_or_model_reference',
    'eval_is_structured_set',
    'eval_is_unique_expression',
    'eval_has_explicit_conditions_for_single_action',
    'eval_is_structured_statement'
]

# Print function names
eval_config = {}
for name, func in functions:
    if (name.startswith("eval")) and (name not in exclude_funcs):
        docstring = inspect.getdoc(func)
        match = re.search(pattern='R\d+', string=str(docstring))
        rule_number = match.group() if match else ''
        #print(name, rule_number)
        eval_config[name] = func

eval_config

{'eval_acronym_consistency': <function src.components.prompteval.eval_acronym_consistency(text: str, acronym_glossary: dict = {}) -> bool>,
 'eval_avoid_pronouns': <function src.components.prompteval.eval_avoid_pronouns(text: str) -> bool>,
 'eval_avoids_absolutes': <function src.components.prompteval.eval_avoids_absolutes(text: str) -> bool>,
 'eval_avoids_parentheses': <function src.components.prompteval.eval_avoids_parentheses(text: str) -> bool>,
 'eval_avoids_purpose_phrases': <function src.components.prompteval.eval_avoids_purpose_phrases(text: str) -> bool>,
 'eval_avoids_vague_terms': <function src.components.prompteval.eval_avoids_vague_terms(text: str) -> bool>,
 'eval_consistent_terms_and_units': <function src.components.prompteval.eval_consistent_terms_and_units(text: str, glossary_terms: list = [None], glossary_units: list = [None], allow_synonyms: bool = False) -> bool>,
 'eval_correct_punctuation': <function src.components.prompteval.eval_correct_punctuation(text: str) -

In [ ]:
def call_evals(df: pd.DataFrame, col: str, eval_config: dict = eval_config) -> pd.DataFrame:
    """
    Run evaluations for each row in the DataFrame.
    
    Args:
        df: DataFrame containing requirements
        col: Column containing requirement text
        
    Returns:
        DataFrame with evaluation results
    """
    result_df = df.copy()
    
    # Run evaluations for each row
    for idx, row in result_df.iterrows():
        for eval_name, eval_func in eval_config.items():
            eval_result = eval_func(row[col])
            result_df.loc[idx, eval_name] = pe.convert_bool_to_ohe(eval_result) # type: ignore
    return result_df

In [32]:
# Load requirements
df = pd.read_excel('../src/data/demo_dataset.xlsx')
requirements = list(df['Requirement'].values)
requirements

['The Disputes System shall record the name of the user and the date for any activity that creates or modifies the disputes case in the system.  A detailed history of the actions taken on the case  including the date and the user that performed the action must be maintained for auditing purposes.',
 'The WCS system shall use appropriate nomenclature and terminology as defined by the Corporate Community Grants organization. All interfaces and reports will undergo usability tests by CCR users.',
 ' The system will notify affected parties when changes occur affecting clinicals  including but not limited to clinical section capacity changes  and clinical section cancellations.',
 'Application testability DESC: Test environments should be built for the application to allow testing of the applications different functions.',
 'The product shall be platform independent.The product shall enable access to any type of development environment and platform.']

In [33]:
# Call evals
eval_df = call_evals(df, 'Requirement', eval_config)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_dat

In [34]:
eval_df

,Unnamed: 0,Requirement_#,Requirement,eval_acronym_consistency,eval_avoid_pronouns,eval_avoids_absolutes,eval_avoids_parentheses,eval_avoids_purpose_phrases,eval_avoids_vague_terms,eval_consistent_terms_and_units,...,eval_is_active_voice,eval_is_independent_of_heading,eval_is_singular_statement,eval_is_solution_free,eval_logical_expressions,eval_no_oblique_symbol,eval_terms_are_defined,eval_uses_abbreviations,eval_uses_not,eval_uses_universal_qualification
0,0,0,The Disputes System shall record the name of t...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
1,1,1,The WCS system shall use appropriate nomenclat...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,2,2,The system will notify affected parties when ...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
3,3,3,Application testability DESC: Test environment...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
4,4,4,The product shall be platform independent.The ...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [ ]:
# Example usage
async def main(requirements):
    # Initialize the client
    # Load config settings
    DOT_ENV = dotenv_values("../.env")
    openai_api_key = str(DOT_ENV['OPENAI_API_KEY'])
    openai_api_key_secret = SecretStr(openai_api_key)
    openai_client = RateLimitOpenAIClient(api_key=openai_api_key_secret.get_secret_value())
    
    # Example of making multiple concurrent requests
    tasks = []
    for i, requirement in enumerate(requirements):
        tasks.append(
            openai_client.chat_completion(
                model="gpt-4o-mini",
                messages=[
                    {"role":"system", "content": 'You are a meticulous requirements analyst tasked with verifying the quality and clarity of a given requirement based on a detailed checklist.'},
                    {"role":"user", "content": f'Given the following requirement:\n\n"""\n{requirement}\n"""\n\nPlease systematically assess the requirement against each of the following criteria, providing a clear answer (Yes/No) and a concise explanation for each:\n\n1. Is the requirement clearly stated, avoiding ambiguous or vague terms?\n2. Can all readers (technical and non-technical stakeholders) understand the requirement without confusion?\n3. Is it written in plain, simple language with no jargon or undefined acronyms?\n4. Does the requirement use active voice and a positive statement (e.g., \'The system shall…\')?\n5. Does it avoid subjective words such as \'user-friendly\', \'fast\', or \'optimal\'?\n6. Is the requirement phrased as a single, atomic statement (not combining multiple requirements)?\n7. Does the requirement address a single capability or attribute?\n8. Avoid compound requirements that include more than one functionality or condition.\n9. Split complex requirements into multiple focused ones if necessary.\n10. Is the requirement stated in such a way that it can be verified through test, inspection, or analysis?\n11. Are the acceptance criteria or measurable parameters clearly indicated or implied?\n12. Could a tester or analyst objectively determine if the requirement is met or not?\n13. Does the requirement use terminology consistent with the rest of the documentation?\n14. Are key terms defined somewhere in a glossary or within the document?\n15. Is there any conflicting wording when compared to other requirements?\n16. Is the requirement traceable back to a stakeholder need, higher-level system requirement, or project objective?\n17. Does the requirement add value to the system, or is it redundant or unnecessary?\n18. Is the rationale for the requirement clear or documented (if applicable)?\n19. Does the requirement cover all relevant conditions and constraints (e.g., operating environment, performance parameters)?\n20. Is it realistically implementable within project constraints (time, cost, technology)?\n\nProvide your answer in a numbered list with each item corresponding to the checklist criteria.'}
                ],
                
            )
        )
    
    # Wait for all requests to complete
    responses = await asyncio.gather(*tasks)
    
    # Process responses
    for i, response in enumerate(responses):
        print(f"\nResponse {i+1}:")
        print(response.choices[0].message.content)

asyncio.run(main(requirements))